In [15]:
import os
from minio import Minio
import requests

In [ ]:
client = Minio("minio-service.kubeflow.svc.cluster.local:9000",
    access_key="minio",
    secret_key="minio123",
    secure=False,           
)

In [ ]:
type(client)

In [ ]:
bucket_name = "newtiledb"

In [ ]:
def upload_files(bucket_name, file_location, client):
    found = False  # Initialize 'found' before the try block
    try:
        found = client.bucket_exists(bucket_name)
    except Exception as e:
        print("error trying to search for MinIO Bucket:", e)
        return  # Return early since we cannot proceed without knowing if the bucket exists

    if not found:
        try:
            client.make_bucket(bucket_name)
            print("Created bucket", bucket_name)
        except Exception as e:
            print("Failed to create bucket:", e)
            return  # Return early since we cannot proceed if the bucket cannot be created
    else:
        print("Bucket", bucket_name, "exists, we won't attempt to create one")
        
    # Ensure the directory exists
    if not os.path.isdir(file_location):
        print(f"The directory {file_location} does not exist.")
        return

    # Iterate through all files in the directory
    for file_name in os.listdir(file_location):
        # Construct the full file path
        source_file = os.path.join(file_location, file_name)
        # Check if it's a file and not a directory
        if os.path.isfile(source_file):
            try:
                # Upload the file
                client.fput_object(bucket_name, file_name, source_file)
                print(f"Successfully uploaded {file_name} to bucket {bucket_name}.")
            except Exception as e:
                print(f"Failed to upload {file_name}: {e}")


In [ ]:
upload_files(bucket_name,"documentation",client)

In [ ]:
import requests

data = {
  "instances": [{
    "input": "what are vectors",
    "num_docs": 4  # number of documents to retrieve
  }]
}

URL = "http://vectorstore-predictor.christensenc3526.svc.cluster.local/v1/models/vectorstore:predict"  # Adjust path as necessary

response = requests.post(URL, json=data, verify=False)  # 'verify=False' for self-signed certs
print(response)
print(response.json())
print(response.text)